In [1]:
from bs4 import BeautifulSoup
from slugify import slugify
import pandas as pd
import requests
import csv

In [2]:
# url = 'https://stats.sharksice.timetoscore.com/display-stats.php?league=27'


# Get the season values


url = 'https://stats.sharksice.timetoscore.com/display-stats.php?league=27'
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
select_element = soup.find('select')
options = select_element.find_all('option')
seasons = {}
for option in options:
    text = option.get_text()
    value = option['value']
    seasons[value] = text  # Add to the dictionary

season_urls = []
for season in seasons:
    season_url = f'https://stats.sharksice.timetoscore.com/display-stats.php?league=27&season={season}'
    season_urls.append(season_url)

print(seasons)


{'0': 'Current', '56': 'Winter 2023', '55': 'Fall 2022', '54': 'Summer 2022', '52': 'Winter 2022', '51': 'Fall 2021', '50': 'Summer 2021', '48': 'Spring 2021', '46': 'Fall 2019', '45': 'Spring 2019', '43': 'Fall 2018', '42': 'Spring 2018', '40': 'Fall 2017', '39': 'Spring 2017', '37': 'Fall 2016', '35': 'Spring 2016', '33': 'Fall 2015', '32': 'Spring 2015', '30': 'Fall 2014', '29': 'Spring 2014', '27': 'Fall 2013', '25': 'Spring 2013', '23': 'Fall 2012', '22': 'Spring 2012', '20': 'Winter 2011', '19': 'Spring 2011', '17': 'Winter 2010', '16': 'Spring 2010', '15': 'Winter 2009', '14': 'Spring 2009', '13': 'Winter 2008', '12': 'Spring 2008', '11': 'Winter 2007', '10': 'Spring 2007'}


In [3]:
url = "https://stats.sharksice.timetoscore.com/display-schedule?team=4622&season=56&league=27&stat_class=1"
def extract_stats(url, player):
    response = requests.get(url, player)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find_all('table')

    
    target_column_text = player
    
    # Iterate through the rows of the table

    season_stats = []
    if len(table) > 0:
        for row in table[1].find_all('tr'):
            columns = row.find_all('td')  # Extract the columns within the row
            if len(columns) > 0:  # Ensure there are columns in the row
                # Assuming the specific column you're interested in is the first column (index 0)
                column_text = columns[0].get_text()
                if target_column_text in column_text:
                    season_stats.append(seasons[season])
                    season_stats.append(teams_dict[team])
                    season_stats.append(row.findAll('td')[2].contents.pop(0))
                    season_stats.append(row.findAll('td')[3].contents.pop(0))
                    season_stats.append(row.findAll('td')[4].contents.pop(0))
                    season_stats.append(row.findAll('td')[25].contents.pop(0))
                    season_stats.append(row.findAll('td')[22].contents.pop(0))
                    season_stats.append(row.findAll('td')[5].contents.pop(0))
                    season_stats.append(row.findAll('td')[6].contents.pop(0))
                    season_stats.append(row.findAll('td')[7].contents.pop(0))
                    season_stats.append(row.findAll('td')[8].contents.pop(0))
                    season_stats.append(row.findAll('td')[9].contents.pop(0))
                    season_stats.append(row.findAll('td')[10].contents.pop(0))
    if len(season_stats) > 0:
        # print(season_stats)
        return season_stats


# Season, Team, Games Played, Goals, Assists, Points, PIMs


In [9]:
players = ["Aaron Thompson",
"ALEX PLASENCIA",
"Blair Menard",
"Brandon Paluzzi",
"BRENDAN BURKE",
"BRIAN FLANIGAN JR",
"BRIAN MCCORMACK",
"Caleb Astey",
"Chris Matthews",
"Christopher Hirneisen",
"CHRISTOPHER MCALLISTER",
"David Raber",
"David Rivera",
"Erik Contreras",
"Evan Borzilleri",
"GINA SAWAYA",
"Jack Levy",
"Jason Moseley",
"Jeffrey Grossmann",
"Kevin Pederson",
"Kyle Bentancourt",
"Marco Pacheco",
"Mark Zegarelli",
"Maximillian Mueller",
"Michael Dearborn",
"Michael Dobson",
"Mike Dobson",
"Nicole Ketterer",
"Paul Lundquist",
"Paul Mullen",
"Pierre Lechene",
"ROBERT KERSHAW",
"Robert Price",
"Tess Nogles",
"Thom Bruce",
"Timothy Lewis",
"Tony Lee",
"VLADISLAV TEPLITSKY"]



In [10]:
teams_dict = {1843: "Gang Green 1", 692: "Gang Green 2", 4622: "Gang Green 3", 4818: "Gang green 4"}
base_url = "https://stats.sharksice.timetoscore.com/display-schedule?"
season_keys = seasons.keys()
team_keys = teams_dict.keys()



for player in players:
    player_stats = []
    for season in season_keys:
        for team in team_keys:
            url = f'https://stats.sharksice.timetoscore.com/display-schedule?team={team}&season={season}'
            if extract_stats(url, player) is not None:
                # print(extract_stats(url, player))
                player_stats.append(extract_stats(url, player))
            # writer.writerow(extract_stats(url, player))

    
    # print(player_stats)

    csv_file = f'data/{slugify(player)}.csv'
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Season', 'Team', 'GP', 'G', 'A', 'P', 'PIMs', 'PPG', 'PPA', 'SHG', 'SHA', 'GWG', 'GWA'])
        writer.writerows(player_stats)
        print(f'{player} file written')
            
            
    clean = pd.read_csv(f'{csv_file}')
    df = pd.DataFrame(clean)
    # print(df)
    df.loc['Total',:] = df.sum(axis=0,numeric_only = True)
    df.to_csv(f'clean/{slugify(player)}.csv', index=False)
    

Aaron Thompson file written
ALEX PLASENCIA file written
Blair Menard file written
Brandon Paluzzi file written
BRENDAN BURKE file written
BRIAN FLANIGAN JR file written
BRIAN MCCORMACK file written
Caleb Astey file written
Chris Matthews file written
Christopher Hirneisen file written
CHRISTOPHER MCALLISTER file written
David Raber file written
David Rivera file written
Erik Contreras file written
Evan Borzilleri file written
GINA SAWAYA file written
Jack Levy file written
Jason Moseley file written
Jeffrey Grossmann file written
Kevin Pederson file written
Kyle Bentancourt file written
Marco Pacheco file written
Mark Zegarelli file written
Maximillian Mueller file written
Michael Dearborn file written
Michael Dobson file written
Mike Dobson file written
Nicole Ketterer file written
Paul Lundquist file written
Paul Mullen file written
Pierre Lechene file written
ROBERT KERSHAW file written
Robert Price file written
Tess Nogles file written
Thom Bruce file written
Timothy Lewis file wri

In [59]:
for player in players:
    slug = slugify(player)
    data = pd.read_csv(f'data/{slug}.csv')
    df = pd.DataFrame(data)
    df.loc['Total',:] = df.sum(axis=0,numeric_only = True)
    df.to_csv(f'clean/{slug}.csv', index=False)
    print(f'Converted {player}')

Converted Mike Dobson


In [22]:
import pandas as pd
markz = pd.read_csv("data/mark-zegarelli.csv")
df = pd.DataFrame(markz)
# print(df)
df.loc['Total',:] = df.sum(axis=0,numeric_only = True)
df
# df.to_csv('your.csv', index=False)


,Season,Team,Games Played,Goals,Assists,Points,PIMs,PPG,PPA,SHG,SHA,GWG,GWA
0,Current,Gang Green 3,6.0,2.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Current,Gang green 4,7.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Winter 2023,Gang Green 3,7.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Fall 2022,Gang Green 3,10.0,2.0,3.0,5.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,Summer 2022,Gang Green 3,8.0,4.0,1.0,5.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0
5,Winter 2022,Gang Green 3,10.0,7.0,3.0,10.0,4.0,1.0,0.0,0.0,0.0,2.0,0.0
6,Fall 2021,Gang Green 3,9.0,4.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Fall 2019,Gang Green 3,17.0,0.0,6.0,6.0,4.0,0.0,1.0,0.0,0.0,0.0,3.0
Total,NaN,NaN,74.0,20.0,20.0,40.0,12.0,2.0,3.0,0.0,0.0,4.0,3.0
